In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.model_selection import cross_val_score, KFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer, StandardScaler, RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold, SelectKBest, mutual_info_regression, SelectFromModel
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_csv("../data/raw/numerical.csv")

In [3]:
df.head()

,date,Tax Effect Of Unusual Items,Tax Rate For Calcs,Normalized EBITDA,Total Unusual Items,Total Unusual Items Excluding Goodwill,Net Income From Continuing Operation Net Minority Interest,Reconciled Depreciation,Reconciled Cost Of Revenue,EBITDA,...,militaryContract,nuclear,pesticides,palmOil,coal,tobacco,percentAumCovered,sustainRank,responsible,sustainScore
0,2024-12-31,0.0,0.237,1.690800e+09,NaN,NaN,1.150600e+09,175400000.0,4.144100e+09,1.690800e+09,...,False,False,False,False,False,False,NaN,NaN,NaN,NaN
1,2023-12-31,0.0,0.241,1.710100e+09,NaN,NaN,1.155000e+09,177300000.0,3.992200e+09,1.710100e+09,...,False,False,False,False,False,False,NaN,NaN,NaN,NaN
2,2022-12-31,0.0,0.245,1.630900e+09,NaN,NaN,1.086900e+09,176600000.0,3.764800e+09,1.630900e+09,...,False,False,False,False,False,False,NaN,NaN,NaN,NaN
3,2021-12-31,0.0,0.234,1.388200e+09,NaN,NaN,9.250000e+08,170700000.0,3.233700e+09,1.388200e+09,...,False,False,False,False,False,False,NaN,NaN,NaN,NaN
4,2020-12-31,NaN,NaN,NaN,1400000.0,1400000.0,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,NaN,NaN,NaN,NaN


In [ ]:
front_columns = ['symbol', 'date', 'companySize', 'region', 'totalEsg', 'shortName', 'longName', 'marketCap', 'cumulativeMarketCapPercentage']

all_columns = df.columns.tolist()
new_column_order = front_columns + [col for col in all_columns if col not in front_columns] 

# Reindex the DataFrame with the new column order
df = df[new_column_order]


In [5]:
df.head()

,symbol,date,companySize,region,totalEsg,shortName,longName,marketCap,cumulativeMarketCapPercentage,ratingYear,...,militaryContract,nuclear,pesticides,palmOil,coal,tobacco,percentAumCovered,sustainRank,responsible,sustainScore
0,FAST,2024-12-31,Mid-Cap,north_america,25.04,Fastenal Company,Fastenal Company,4.367976e+10,70.076539,2025,...,False,False,False,False,False,False,NaN,NaN,NaN,NaN
1,FAST,2023-12-31,Mid-Cap,north_america,25.04,Fastenal Company,Fastenal Company,4.367976e+10,70.076539,2025,...,False,False,False,False,False,False,NaN,NaN,NaN,NaN
2,FAST,2022-12-31,Mid-Cap,north_america,25.04,Fastenal Company,Fastenal Company,4.367976e+10,70.076539,2025,...,False,False,False,False,False,False,NaN,NaN,NaN,NaN
3,FAST,2021-12-31,Mid-Cap,north_america,25.04,Fastenal Company,Fastenal Company,4.367976e+10,70.076539,2025,...,False,False,False,False,False,False,NaN,NaN,NaN,NaN
4,FAST,2020-12-31,Mid-Cap,north_america,25.04,Fastenal Company,Fastenal Company,4.367976e+10,70.076539,2025,...,False,False,False,False,False,False,NaN,NaN,NaN,NaN


In [6]:
df.shape

(4646, 392)

In [7]:
df['symbol'].nunique()

1012

In [8]:
# Create a latest_date feature to align different EOY financial dates
df['latest_date'] = df.groupby('symbol')['date'].transform('max')

In [9]:
df['latest_date'].value_counts()

latest_date
2024-12-31    2603
2024-03-31     925
2023-12-31     444
2024-06-30     180
2024-09-30     156
2025-01-31      74
2024-02-29      58
2024-08-31      44
2024-07-31      34
2024-10-31      33
2024-01-31      28
2024-05-31      25
2024-04-30      15
2023-03-31      11
2022-12-31      11
2024-11-30       5
Name: count, dtype: int64

In [10]:
df['latest_year'] = df['latest_date'].apply(lambda x: x.split('-')[0])

In [ ]:
sentiments = pd.read_csv("../data/processed/news_sentiment.csv")
sentiments.head()

In [ ]:
sentiments['symbol'].nunique()

In [ ]:
# filtered_df = latest_entries[latest_entries['symbol'].isin(sentiments['symbol'])]
# filtered_df[financial_features].head()
# filtered_df.columns.tolist()

## Data Cleaning

### Assessing Missingness Patterns

In [11]:
df.shape

(4646, 394)

In [12]:
# Drop features with more than 70% missing values, as they offer too little information to be imputed
pct_null = df.isnull().mean()
df = df.drop(columns=pct_null[pct_null > 0.7].index.tolist())
df.shape

(4646, 233)

### Assessing constant features

Features with zero or very low variance provide little information and can be removed.

In [13]:
# Calculate variance for each numerical column
feature_variance = df.var(numeric_only=True)
feature_variance.sort_values(ascending=True).head(10)

maxAge                             0.000000
Tax Rate For Calcs                 0.006051
ratingYear                         0.513186
highestControversy                 1.181631
ratingMonth                        4.228918
governanceScore                    8.557575
socialScore                       12.425297
environmentScore                  33.442564
totalEsg                          69.034408
cumulativeMarketCapPercentage    368.737453
dtype: float64

In [14]:
vt = VarianceThreshold(threshold=0.1)
vt.fit(df.select_dtypes(include=[np.number]).fillna(0))
constant_feats = df.select_dtypes(include=[np.number]).columns[~vt.get_support()]
constant_feats

Index(['Tax Rate For Calcs', 'maxAge'], dtype='object')

In [15]:
df = df.drop(columns=constant_feats)

In [16]:
df.shape

(4646, 231)

### Missing Value Imputation

Step 1: Company‑level ffill/bfill (and/or rolling)

Step 2: Peer‑group imputation (region + company size + year)

Step 3: Global/statistical imputer (median) for any stragglers

In [ ]:
# defining columns that should be imputed
df_sorted = df.sort_values(by=['symbol', 'latest_year'], ascending=True)
meta_numeric = ['date', 'latest_date', 'latest_year', 'totalEsg', 'environmentScore', 'socialScore', 'governanceScore']

cols_to_impute = df_sorted.select_dtypes(include=np.number).columns.tolist()
# remove identifiers/targets if they are numeric and shouldn't be imputed this way
cols_to_impute = [col for col in cols_to_impute if col not in meta_numeric]

In [18]:
# Step 1: company-level historical imputation
df_filled = df_sorted.copy()
df_filled[cols_to_impute] = df_filled.groupby('symbol')[cols_to_impute].ffill()
df_filled[cols_to_impute] = df_filled.groupby('symbol')[cols_to_impute].bfill()

In [19]:
len(df_filled[cols_to_impute].columns[df_filled[cols_to_impute].isnull().any()].tolist())

161

In [20]:
# Step 2
cols_to_impute_final = [
    col for col in df_filled.select_dtypes(include=np.number).columns.tolist()
    if col not in meta_numeric
    and df_filled[col].isnull().any() # Only consider columns with NaNs remaining
]

In [21]:
df_filled.groupby(['region', 'companySize', 'latest_year']).count().head()

symbol  date  totalEsg  \
region              companySize latest_year                           
east_asia_n_pacific Large-Cap   2022              3     3         3   
                                2023             48    48        48   
                                2024            204   204       204   
                    Mid-Cap     2023            104   104       104   
                                2024            332   332       332   

                                             shortName  longName  marketCap  \
region              companySize latest_year                                   
east_asia_n_pacific Large-Cap   2022                 3         3          3   
                                2023                48        48         48   
                                2024               204       204        204   
                    Mid-Cap     2023               104       104        104   
                                2024               332       332        332   

                                             cumulativeMarketCapPercentage  \
region              companySize latest_year                                  
east_asia_n_pacific Large-Cap   2022                                     3   
                                2023                                    48   
                                2024                                   204   
                    Mid-Cap     2023                                   104   
                                2024                                   332   

                                             ratingYear  ratingMonth  \
region              companySize latest_year                            
east_asia_n_pacific Large-Cap   2022                  3            3   
                                2023                 48           48   
                                2024                204          204   
                    Mid-Cap     2023                104          104   
                                2024                332          332   

                                             Tax Effect Of Unusual Items  ...  \
region              companySize latest_year                               ...   
east_asia_n_pacific Large-Cap   2022                                   3  ...   
                                2023                                  48  ...   
                                2024                                 204  ...   
                    Mid-Cap     2023                                 104  ...   
                                2024                                 332  ...   

                                             furLeather  gambling  gmo  \
region              companySize latest_year                              
east_asia_n_pacific Large-Cap   2022                  3         3    3   
                                2023                 48        48   48   
                                2024                204       204  204   
                    Mid-Cap     2023                104       104  104   
                                2024                332       332  332   

                                             militaryContract  nuclear  \
region              companySize latest_year                              
east_asia_n_pacific Large-Cap   2022                        3        3   
                                2023                       48       48   
                                2024                      204      204   
                    Mid-Cap     2023                      104      104   
                                2024                      332      332   

                                             pesticides  palmOil  coal  \
region              companySize latest_year                              
east_asia_n_pacific Large-Cap   2022                  3        3     3   
                                2023                 48       48    48   
                                2024   

In [22]:
for col in cols_to_impute_final:
    # Use transform with median. It aligns results back to the original DataFrame index.
    df_filled[col] = df_filled.groupby(['region', 'companySize', 'latest_year'])[col].transform(lambda x: x.fillna(x.median()))

In [23]:
remaining_nan_counts = df_filled[cols_to_impute_final].isnull().sum()
cols_still_nan = remaining_nan_counts[remaining_nan_counts > 0].index.tolist()

In [24]:
remaining_nan_counts[remaining_nan_counts > 0].sort_values(ascending=True).head(10)

Other Properties                        2
Net Business Purchase And Sale          2
Net Investment Purchase And Sale        2
Goodwill And Other Intangible Assets    3
Operating Cash Flow                     3
Other Non Cash Items                    3
Change In Payable                       3
Change In Working Capital               3
Depreciation And Amortization           3
Other Intangible Assets                 3
dtype: int64

In [25]:
# Step 3
for col in cols_still_nan:
    global_median_val = df_filled[col].median()
    df_filled[col] = df_filled[col].fillna(global_median_val)

    df_filled[cols_to_impute_final].isnull().sum().sort_values(ascending=True).head(10)

In [26]:
df_filled.sort_index()

,symbol,date,companySize,region,totalEsg,shortName,longName,marketCap,cumulativeMarketCapPercentage,ratingYear,...,gambling,gmo,militaryContract,nuclear,pesticides,palmOil,coal,tobacco,latest_date,latest_year
0,FAST,2024-12-31,Mid-Cap,north_america,25.04,Fastenal Company,Fastenal Company,4.367976e+10,70.076539,2025,...,False,False,False,False,False,False,False,False,2024-12-31,2024
1,FAST,2023-12-31,Mid-Cap,north_america,25.04,Fastenal Company,Fastenal Company,4.367976e+10,70.076539,2025,...,False,False,False,False,False,False,False,False,2024-12-31,2024
2,FAST,2022-12-31,Mid-Cap,north_america,25.04,Fastenal Company,Fastenal Company,4.367976e+10,70.076539,2025,...,False,False,False,False,False,False,False,False,2024-12-31,2024
3,FAST,2021-12-31,Mid-Cap,north_america,25.04,Fastenal Company,Fastenal Company,4.367976e+10,70.076539,2025,...,False,False,False,False,False,False,False,False,2024-12-31,2024
4,FAST,2020-12-31,Mid-Cap,north_america,25.04,Fastenal Company,Fastenal Company,4.367976e+10,70.076539,2025,...,False,False,False,False,False,False,False,False,2024-12-31,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4641,8411.T,2024-03-31,Mid-Cap,east_asia_n_pacific,24.36,MIZUHO FINANCIAL GROUP,"Mizuho Financial Group, Inc.",1.064007e+13,79.513237,2025,...,False,False,False,False,False,False,False,False,2024-03-31,2024
4642,8411.T,2023-03-31,Mid-Cap,east_asia_n_pacific,24.36,MIZUHO FINANCIAL GROUP,"Mizuho Financial Group, Inc.",1.064007e+13,79.513237,2025,...,False,False,False,False,False,False,False,False,2024-03-31,2024
4643,8411.T,2022-03-31,Mid-Cap,east_asia_n_pacific,24.36,MIZUHO FINANCIAL GROUP,"Mizuho Financial Group, Inc.",1.064007e+13,79.513237,2025,...,False,False,False,False,False,False,False,False,2024-03-31,2024
4644,8411.T,2021-03-31,Mid-Cap,east_asia_n_pacific,24.36,MIZUHO FINANCIAL GROUP,"Mizuho Financial Group, Inc.",1.064007e+13,79.513237,2025,...,False,False,False,False,False,False,False,False,2024-03-31,2024


In [27]:
df = df_filled

In [28]:
df.columns[df.isnull().any()].tolist()

['shortName',
 'longName',
 'environmentScore',
 'socialScore',
 'governanceScore',
 'relatedControversy',
 'peerGovernancePerformance',
 'peerSocialPerformance',
 'peerEnvironmentPerformance',
 'adult',
 'alcoholic',
 'animalTesting',
 'controversialWeapons',
 'smallArms',
 'furLeather',
 'gambling',
 'gmo',
 'militaryContract',
 'nuclear',
 'pesticides',
 'palmOil',
 'coal',
 'tobacco']

## Feature Engineering

### Feature Creation

Creating different financial ratios

In [29]:
def safe_div(numer, denom):
    """Divide two Series, returning NaN where denom is zero or NaN."""
    return numer.div(denom).replace([np.inf, -np.inf], np.nan)

# Defining the ratios to be calculated and their required features
financial_ratios = {
    'profit_margin': (['Net Income'], 'Operating Revenue'),
    'ROA': (['Net Income'], 'Total Assets'),
    'ROE': (['Net Income Common Stockholders'], 'Common Stock Equity'),
    'debt_to_equity': (['Total Debt'], 'Total Equity Gross Minority Interest'),
    'interest_coverage': (['EBIT'], 'Interest Expense'),
    'current_ratio': (['Current Assets'], 'Current Liabilities'),
    'quick_ratio': (['Current Assets', 'Inventory'], 'Current Liabilities'),
    # below are ratios suggested by D'amato et al. (2021)
    'sales_to_assets': [['Total Revenue'], 'Total Assets'],
    'EBIT_to_sales': [['EBIT'], 'Total Revenue'],
    'dividend_yield': [['Cash Dividends Paid'], 'marketCap'],
    'net_income_to_sales': [['Net Income'], 'Total Revenue'],
    'liquidity_ratio': [['Current Assets'], 'Current Liabilities'],
    'solvency_ratio': [['Total Debt'], 'Total Assets'],
    'price_to_earnings': [['marketCap', 'Ordinary Shares Number'], 'Diluted EPS'],
}

In [30]:
# Filter the data to include only the latest entries for each symbol
df_latest = df[df['date'] == df['latest_date']]
df_latest = df_latest.drop(columns=['latest_date'])

In [ ]:
# Handle the creation of financial ratios
for name, (num_cols, den_col) in financial_ratios.items():
    if den_col in df_latest.columns and all(c in df_latest.columns for c in num_cols):
        print(f"Calculating {name}...")
        # handles the cases where the numerator is a list of columns
        if len(num_cols) > 1:
            if name == 'price_to_earnings':
                numerator = safe_div(df_latest[num_cols[0]], df_latest[num_cols[1]])
            elif name == 'quick_ratio':
                numerator = df_latest[num_cols[0]] - df_latest[num_cols[1]]
            else:
                print("Undefined numerator for this ratio")
                continue
        else:   
            numerator = df_latest[num_cols].sum(axis=1)
        df_latest[name] = safe_div(numerator, df_latest[den_col])

Calculating profit_margin...
Calculating ROA...
Calculating ROE...
Calculating debt_to_equity...
Calculating interest_coverage...
Calculating current_ratio...
Calculating quick_ratio...
Calculating sales_to_assets...
Calculating EBIT_to_sales...
Calculating dividend_yield...
Calculating net_income_to_sales...
Calculating liquidity_ratio...
Calculating solvency_ratio...
Calculating price_to_earnings...


C:\Users\Mishaal\AppData\Local\Temp\ipykernel_7096\3256737879.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_latest[name] = safe_div(numerator, df_latest[den_col])
C:\Users\Mishaal\AppData\Local\Temp\ipykernel_7096\3256737879.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_latest[name] = safe_div(numerator, df_latest[den_col])
C:\Users\Mishaal\AppData\Local\Temp\ipykernel_7096\3256737879.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, w

In [32]:
df_latest[financial_ratios.keys()].head()

,profit_margin,ROA,ROE,debt_to_equity,interest_coverage,current_ratio,quick_ratio,sales_to_assets,EBIT_to_sales,dividend_yield,net_income_to_sales,liquidity_ratio,solvency_ratio,price_to_earnings
4570,0.090955,0.105362,0.175045,0.064407,134.021323,1.549352,1.089000,1.158404,0.126588,-0.058161,0.090955,1.549352,0.038774,3.867953
3894,0.354687,0.007948,0.123894,1.260432,1.363844,1.653200,1.365704,0.022408,1.531121,-0.010776,0.354687,1.653200,0.080326,77.609565
1521,0.298954,0.165105,0.267764,0.344340,18.755469,1.693496,1.160202,0.552274,0.381167,-0.006006,0.298954,1.693496,0.212357,-15.076853
4618,0.071487,0.036865,0.115164,1.311580,40.383200,0.809155,0.560242,0.515691,0.104051,-0.077791,0.071487,0.809155,0.464250,3.987367
1474,0.022132,0.016733,0.031363,0.451264,3.632031,2.079955,1.432030,0.759280,0.047195,-0.036930,0.022038,2.079955,0.265008,13.002283


In [33]:
df_latest.columns[df_latest.isnull().any()].tolist()

['shortName',
 'longName',
 'environmentScore',
 'socialScore',
 'governanceScore',
 'relatedControversy',
 'peerGovernancePerformance',
 'peerSocialPerformance',
 'peerEnvironmentPerformance',
 'adult',
 'alcoholic',
 'animalTesting',
 'controversialWeapons',
 'smallArms',
 'furLeather',
 'gambling',
 'gmo',
 'militaryContract',
 'nuclear',
 'pesticides',
 'palmOil',
 'coal',
 'tobacco',
 'interest_coverage',
 'price_to_earnings']

In [36]:
df_latest.sort_index()

,symbol,date,companySize,region,totalEsg,shortName,longName,marketCap,cumulativeMarketCapPercentage,ratingYear,...,interest_coverage,current_ratio,quick_ratio,sales_to_assets,EBIT_to_sales,dividend_yield,net_income_to_sales,liquidity_ratio,solvency_ratio,price_to_earnings
0,FAST,2024-12-31,Mid-Cap,north_america,25.04,Fastenal Company,Fastenal Company,4.367976e+10,70.076539,2025,...,207.589041,4.674574,2.280454,1.606215,0.200822,-0.020451,0.152478,4.674574,0.103321,37.716505
5,0A2S.IL,2023-12-31,Mid-Cap,europe_n_central_asia,28.97,PDD HOLDINGS INC PDD HOLDINGS A,PDD Holdings Inc.,1.780311e+11,80.769667,2025,...,1635.145429,1.927722,1.927629,0.711447,0.290443,-0.030627,0.242395,1.927722,0.029206,0.785927
9,BXSL,2024-12-31,Small-Cap,north_america,29.32,Blackstone Secured Lending Fund,Blackstone Secured Lending Fund,7.502080e+09,90.670975,2025,...,1.612505,0.238717,-0.309630,0.054279,0.697423,-0.077759,0.949174,0.238717,0.523751,9.799876
14,NED.JO,2024-12-31,Mid-Cap,sub_saharan_africa,14.43,Nedbank Group Ltd,Nedbank Group Limited,1.211992e+11,84.197999,2025,...,0.087673,1.883886,1.469634,0.047586,0.109108,-0.081873,0.271410,1.883886,0.036955,8.296999
19,SCBGF,2024-12-31,Small-Cap,north_america,10.55,SIG GROUP AG,SIG Group AG,7.819702e+09,90.327456,2025,...,3.273463,0.582181,0.405533,0.432880,0.121556,-0.024016,0.058435,0.582181,0.321373,31.965650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4622,ICGUF,2024-03-31,Small-Cap,north_america,12.95,INTERMEDIATE CAPITAL GROUP PLC,Intermediate Capital Group plc,7.619675e+09,90.566796,2025,...,11.358575,1.509065,0.966757,0.101501,0.550815,-0.029319,0.511286,1.509065,0.671764,16.173724
4627,AMGN,2024-12-31,Large-Cap,north_america,22.76,Amgen Inc.,Amgen Inc.,1.648951e+11,45.512649,2025,...,2.460856,1.256764,0.953808,0.363941,0.232288,-0.029303,0.122367,1.256764,0.654395,24.585043
4632,6586.T,2024-03-31,Small-Cap,east_asia_n_pacific,27.34,MAKITA CORP,Makita Corporation,1.363738e+12,92.536957,2025,...,10.421192,4.768070,2.393153,0.702202,0.095512,-0.004163,0.058931,4.768070,0.018840,31.266326
4636,GPK,2024-12-31,Small-Cap,north_america,15.96,Graphic Packaging Holding Compa,Graphic Packaging Holding Company,7.841590e+09,90.307257,2025,...,4.852174,1.462953,0.541251,0.790291,0.126717,-0.015558,0.074713,1.462953,0.470836,11.164277


### Temporal Alignment

Since companies have different fiscal years, the dates have to be aligned accordingly

In [37]:
# Filter out entires that are before 2024
df_latest = df_latest[df_latest['latest_year'] >= '2024']

In [44]:
df_latest['latest_year'].value_counts()

latest_year
2024    878
2025     16
Name: count, dtype: int64

### Feature Scaling

### Correlation Clustering